# ParaView Hello Sphere Test 2
This notebook creates a synthetic data source (a sphere), and creates a polygonal display of it. So far, we have used a single task for the ParaView server (pvserver).

We now demonstrate how to run an MPI-enabled pserver on the allocated node

Tested Wed Aug 24 10:42:45 AM CEST 2022

In [ ]:
from paraview.simple import *

# to turn off some annoying warnings
from vtkmodules.vtkCommonCore import vtkLogger
vtkLogger.SetStderrVerbosity(vtkLogger.VERBOSITY_OFF)
from paraview.modules.vtkRemotingCore import vtkProcessModule

we first launch a parallel MPI job on the node, using a second notebook.

The jupyter client will connect to the ParaView server on the same node

**You must run the notebook called Hello_Sphere-ParaView.2-Helper.ipynb before proceeding**

In [ ]:
Connect("localhost")

In [ ]:
rank = vtkProcessModule.GetProcessModule().GetPartitionId()
nbprocs = servermanager.ActiveConnection.GetNumberOfDataPartitions()
info = GetOpenGLInformation(location=servermanager.vtkSMSession.RENDER_SERVER)

In [ ]:
Version = str(GetParaViewVersion())
if rank == 0:
    print("ParaView Version ", Version)
    print("rank=", rank, "/", nbprocs)
    print("Vendor:   %s" % info.GetVendor())
    print("Version:  %s" % info.GetVersion())
    print("Renderer: %s" % info.GetRenderer())

In [ ]:
view = GetRenderView()
from ipyparaview.widgets import PVDisplay
pvdisp = PVDisplay(view)
w = display(pvdisp)

In [ ]:
sphere = Sphere(ThetaResolution=32, PhiResolution=32)
pidscal = ProcessIdScalars(sphere)

rep = Show(pidscal, view)
ColorBy(rep, 'ProcessId')
processIdLUT = GetColorTransferFunction('ProcessId')
processIdLUT.AnnotationsInitialized = 1
processIdLUT.InterpretValuesAsCategories = 1

IndexedColors = [
    1.0, 1.0, 1.0,
    1.0, 0.0, 0.0,
    0.0, 1.0, 0.0,
    0.0, 0.0, 1.0]

a = []
for i in range(nbprocs):
    a.extend((str(i), str(i)))

processIdLUT.Annotations = a
processIdLUT.IndexedColors = IndexedColors
rep.Representation = "Surface With Edges"

In [ ]:
ResetCamera()
GetActiveCamera().Dolly(1.4)

In [ ]:
# Interact from ipywidgets gives us a simple way to interactively control values
# with a callback function
from ipywidgets import interact, IntSlider

# set the Theta and Phi resolution and trigger a pipeline update
def Sphere_resolution(res):
    sphere.ThetaResolution = sphere.PhiResolution = res
    sphere.UpdatePipeline()
    
i = interact(Sphere_resolution, res=IntSlider(min=3, max=48, step=1, value=12))

In [ ]:
def SaveImage(filename):
  from vtk import vtkPNGWriter
  img_writer = vtkPNGWriter()
  img_writer.SetInputConnection(pvdisp.w2i.GetOutputPort())
  img_writer.SetFileName(filename)
  img_writer.Write()

# save animage on the client-side
#SaveImage("/users/jfavre/foo2.png")

We can cleanly close the connection with the following command

In [ ]:
Disconnect()